In [112]:
import xmltodict
from glob import glob
from tqdm import tqdm
from fastcore.xtras import save_pickle
import pandas as pd 

In [27]:
example_xml = '/Users/kevinmaikjablonka/Downloads/5104873/applications/2016/I20160728.xml'

In [28]:
with open(example_xml, 'r') as f:
    xml_dict = xmltodict.parse(f.read())

In [43]:
def has_product_yield(x):
    try:
        return any([y['amount']['@dl:propertyType']=='PERCENTYIELD' for y in x['productList']['product']])
    except KeyError:
        return False
    except:
        return False

def has_product_amount(x):
    try:
        return any([y['amount']['@dl:propertyType']=='MASS' for y in x['productList']['product']])
    except KeyError:
        return False
    except:
        return False

def has_two_elements_in_product_list(x):
    try:
        return len(x['productList']['product'])==2
    except KeyError:
        return False
    except:
        return False

In [30]:
def get_reaction_list(xml_dict):
    return xml_dict['reactionList']['reaction']

In [44]:
reactions_with_yield = [x for x in get_reaction_list(xml_dict) if has_product_yield(x) and has_product_amount(x) and has_two_elements_in_product_list(x)]

In [45]:
len(reactions_with_yield)

15

In [46]:
reactions_with_yield[12]

{'dl:source': {'dl:documentId': 'US20160213657A1',
  'dl:headingText': '1-(f): tert-Butyl (tert-butoxycarbonyl {6-[(pyridin-2-ylsulfonyl)aminomethyl]pyridin-2-yl}amino)acetate',
  'dl:paragraphNum': '0217',
  'dl:paragraphText': '12 mL solution of methylene chloride containing 1.20 g (3.56 mmol) of the tert-butyl [(6-aminomethylpyridin-2-yl)tert-butoxycarbonylamino]acetate obtained in Reference Example 1-(e) and 2.24 mL (16.2 mmol) of triethylamine were added to 14 mL solution of methylene chloride containing 0.640 g (3.60 mmol) of 2-pyridylsulfonyl chloride followed by stirring for 0.5 hours at room temperature. After completion of the reaction, a 5% by weight aqueous potassium hydrogen sulfate solution was added to the reaction solution followed by extracting with methylene chloride. The organic layer was sequentially washed with saturated aqueous sodium bicarbonate solution and saturated aqueous sodium chloride solution, dried with anhydrous magnesium sulfate, and then concentrated 

It probably makes sense to remove the sentences with characterization data as they do not contain any information that we have *before* (or during) the reaction. At best, this only is superfluuos tokens. At worst, it is data leakage.

In [97]:
def remove_characterization_sentences(text): 
    sentences = text.split('. ')
    sentences_without_characterization = []

    for sentence in sentences: 
        if ('NMR' and 'Hz') in sentence: # matching with the unicode delta gives weird results... 
            continue
        if ('MS' and 'm/z') in sentence:
            continue
        sentences_without_characterization.append(sentence)
    return '. '.join(sentences_without_characterization)


In [98]:
def extract_data_from_reaction(reaction): 
    yield_amount = [x['amount']['@dl:normalizedValue'] for x in reaction['productList']['product'] if x['amount']['@dl:propertyType']=='PERCENTYIELD'][0]
    mass_amount = [x['amount']['@dl:normalizedValue'] for x in reaction['productList']['product'] if x['amount']['@dl:propertyType']=='MASS'][0]
    
    # extract the yield text from the dl:reactionActionList
    yield_text = [x['dl:phraseText'] for x in reaction['dl:reactionActionList']['dl:reactionAction'] if x['@action']=='Yield'][0]

    reac_info = {
        'reaction_smiles': reaction['dl:reactionSmiles'],
        'yield': float(yield_amount),
        'mass': float(mass_amount),
        'yield_text': yield_text
    }
    reac_info.update(reaction['dl:source'])
    reac_info['paragraph_without_yield'] = reac_info['dl:paragraphText'].replace(reac_info['yield_text'], '')
    reac_info['paragraph_without_yield_and_charac'] = remove_characterization_sentences(reac_info['paragraph_without_yield'])
    return reac_info

In [99]:
extract_data_from_reaction(reactions_with_yield[12])

{'reaction_smiles': 'C(Cl)Cl.[NH2:4][CH2:5][C:6]1[N:11]=[C:10]([N:12]([CH2:20][C:21]([O:23][C:24]([CH3:27])([CH3:26])[CH3:25])=[O:22])[C:13]([O:15][C:16]([CH3:19])([CH3:18])[CH3:17])=[O:14])[CH:9]=[CH:8][CH:7]=1.C(N(CC)CC)C.[N:35]1[CH:40]=[CH:39][CH:38]=[CH:37][C:36]=1[S:41](Cl)(=[O:43])=[O:42].S([O-])(O)(=O)=O.[K+]>>[C:16]([O:15][C:13]([N:12]([CH2:20][C:21]([O:23][C:24]([CH3:27])([CH3:26])[CH3:25])=[O:22])[C:10]1[CH:9]=[CH:8][CH:7]=[C:6]([CH2:5][NH:4][S:41]([C:36]2[CH:37]=[CH:38][CH:39]=[CH:40][N:35]=2)(=[O:43])=[O:42])[N:11]=1)=[O:14])([CH3:19])([CH3:18])[CH3:17] |f:4.5|',
 'yield': 86.0,
 'mass': 1.46,
 'yield_text': 'to obtain 1.46 g of the title compound in the form of a white solid (yield: 86%)',
 'dl:documentId': 'US20160213657A1',
 'dl:headingText': '1-(f): tert-Butyl (tert-butoxycarbonyl {6-[(pyridin-2-ylsulfonyl)aminomethyl]pyridin-2-yl}amino)acetate',
 'dl:paragraphNum': '0217',
 'dl:paragraphText': '12 mL solution of methylene chloride containing 1.20 g (3.56 mmol) of the t

In [101]:
all_xmls = glob('/Users/kevinmaikjablonka/Downloads/5104873/applications/*/*.xml')

In [102]:
len(all_xmls)

1364

In [105]:
all_reactions = []

for xml in tqdm(all_xmls):
    with open(xml, 'r') as f:
        xml_dict = xmltodict.parse(f.read())
    try:
        reactions = get_reaction_list(xml_dict)
    except Exception as e:
        print(f'Error with {xml}: {e}')
    reactions_with_yield = [x for x in reactions if has_product_yield(x) and has_product_amount(x) and has_two_elements_in_product_list(x)]
    all_reactions.extend(reactions_with_yield)

  4%|▍         | 61/1364 [01:10<21:53,  1.01s/it]

Error with /Users/kevinmaikjablonka/Downloads/5104873/applications/2014/I20140918.xml: 'reaction'


 15%|█▌        | 211/1364 [04:15<22:13,  1.16s/it]

Error with /Users/kevinmaikjablonka/Downloads/5104873/applications/2008/I20080410-SUPP.xml: 'reaction'


 16%|█▌        | 213/1364 [04:15<14:45,  1.30it/s]

...

 99%|█████████▉| 1356/1364 [12:13<00:04,  1.94it/s]

Error with /Users/kevinmaikjablonka/Downloads/5104873/applications/2004/20040415-SUPP.xml: 'reaction'
Error with /Users/kevinmaikjablonka/Downloads/5104873/applications/2004/20040108-SUPP.xml: 'reaction'


100%|█████████▉| 1359/1364 [12:13<00:01,  2.85it/s]

Error with /Users/kevinmaikjablonka/Downloads/5104873/applications/2004/20040318-SUPP.xml: 'reaction'


100%|██████████| 1364/1364 [12:16<00:00,  1.85it/s]


In [106]:
len(all_reactions)

1638

This seems too little ...

In [108]:
save_pickle('all_reactions.pkl', all_reactions)

In [109]:
all_reaction_data = []

for reaction in tqdm(all_reactions):
    try:
        all_reaction_data.append(extract_data_from_reaction(reaction))
    except Exception as e:
        print(f'Error with {reaction}: {e}')

100%|██████████| 1638/1638 [00:00<00:00, 80408.58it/s]

Error with {'dl:source': {'dl:documentId': 'US20130150362A1', 'dl:paragraphNum': '0735', 'dl:paragraphText': 'To a solution of ethyl 1,4-dioxaspiro[4.5]decane-8-carboxylate (7.89 g, 36.8 mmol) in THF (120 mL) was added LDA (1.5 eq) at −78° C. and stirred for 30 min. Then, benzyl chloromethyl ether (10.7 mL, 92.1 mmol) was added dropwise. The resulting mixture was slowly warmed to rt and stirred overnight. The reaction was quenched with H2O and combined with ammonium chloride. All the volatiles were removed. The residue was diluted with EtOAc, washed with H2O and brine, dried over Na2SO4, and concentrated. The resulting brownish oil was dissolved in CH3CN (150 mL) and a solution of CAN (2.02 g 3.68 mmol) in H2O (150 mL) was added. The resulting mixture was heated at 70° C. for 6 h. Organic solvent was removed under reduced pressure. The aqueous residue was extracted with EtOAc (*3) and the combined organic layers were washed brine, dried over Na2SO4 and concentrated. The crude product w

In [110]:
len(all_reaction_data)

1555

In [111]:
save_pickle('all_reaction_data.pkl', all_reaction_data)

In [114]:
df = pd.DataFrame(all_reaction_data)

In [116]:
df.iloc[1553]["paragraph_without_yield_and_charac"]

'The diol 26 (8.17 g, 78.44 mmol) in 12 ml dry toluene is initially introduced into a 50 ml round-bottomed flask equipped with an argon balloon flask and reflux condenser. The solution is stirred and sodium hydride (60% suspension in mineral oil) (1.53 g, 38.25 mmol) is added thereto in portions over a period of 45 min. The suspension is heated under reflux for 3 h under an argon blanketing gas atmosphere. 4-Methoxy-benzyl chloride (5.2 ml, 38.31 mmol) is then added over a period of 30 min and the mixture is boiled under reflux for 18 h. At the end of this period the reaction mixture is cooled to room temperature and poured into 50 ml water. The phases are separated and the aqueous phase is extracted with dichloromethane (3×50 ml). The combined organic phases are dried over Na2SO4 and concentrated under vacuum'

In [118]:
df.iloc[1553]["yield"]

40.0

In [117]:
df.to_csv('../data/USTPO_yields.csv', index=False)